In [1]:
#import statements
import pandas as pd
import numpy as np
import os
import re

In [2]:
# get relative path
os.listdir(os.getcwd())

['.git',
 '.ipynb_checkpoints',
 '1DConvolution_nayana.ipynb',
 'Classification_RF.ipynb',
 'clean_data.csv',
 'clean_data.ipynb',
 'clean_data_cld2withpoly.csv',
 'clean_data_Cldpoly.csv',
 'clean_data_test_balanced_final.csv',
 'clean_data_train_balanced_final.csv',
 'clean_data_with features.csv',
 'clean_data_with_all_features.csv',
 'clean_data_with_all_features.zip',
 'embedding_square.txt',
 'embedding_square_train.txt',
 'Emoticon and Slang dictionary Creation.ipynb',
 'EmoticonLookupTable.txt',
 'Featuredataset.csv',
 'glove.6B.100d.txt',
 'Handcrafted_features.finalML.ipynb',
 'neural_net with parent comment.html',
 'neural_net with parent comment_files',
 'preprocessed_added_features.ipynb',
 'preprocessed_added_features_merged.ipynb',
 'preprocessed_features_classifiers.ipynb',
 'preprocessed_features_classifiers_withembedding.ipynb',
 'README.md',
 'sarcasmdetection_RF.ipynb',
 'Slangdictionary.txt',
 'test-balanced.csv',
 'text_process.ipynb',
 'train-balanced.csv',
 'Unt

In [3]:
header = ['label','comment','parent_comment']
train_clean = pd.read_table('clean_data_train_balanced_final.csv',
                    sep='|', 
                    usecols=[0,1,2],
                    dtype={'label':int,'comment':str},
                    keep_default_na=False)

In [4]:
test_clean = pd.read_table('clean_data_test_balanced_final.csv',
                    sep='|', 
                    usecols=[0,1],
                    dtype={'label':int,'comment':str},
                    keep_default_na=False)

In [5]:
# from sklearn.model_selection import train_test_split

train_comments = train_clean['comment']
train_labels = train_clean['label']
SEED = 100

test_comments = test_clean['comment']
test_labels = test_clean['label']

#split the data into test and train
# comments_train, comments_test, labels_train, labels_test = train_test_split(comments, labels, test_size=.20, random_state=SEED)

In [6]:
# print the number of sarcastic and non sarcastic comments in training set
print ("Train set has total {0} entries with {1:.2f}% non-sarcastic, {2:.2f}% sarcastic".format(len(train_comments),
                                (len(train_comments[train_labels == 0]) / (len(train_comments)*1.))*100,
                                (len(train_comments[train_labels == 1]) / (len(train_comments)*1.))*100))
# print the number of sarcastic and non sarcastic comments in testing set
print ("Test set has total {0} entries with {1:.2f}% non-sarcastic, {2:.2f}% sarcastic".format(len(test_comments),
                                (len(test_comments[test_labels == 0]) / (len(test_comments)*1.))*100,
                                (len(test_comments[test_labels == 1]) / (len(test_comments)*1.))*100))

Train set has total 978039 entries with 49.49% non-sarcastic, 50.51% sarcastic
Test set has total 243572 entries with 49.50% non-sarcastic, 50.50% sarcastic


In [9]:
frames = [train_comments, test_comments]
mearged_comment = pd.concat(frames)
print(mearged_comment.shape)
print(train_comments.shape)
print(test_comments.shape)

(1221611,)
(978039,)
(243572,)


In [9]:
#find maximum sizde of the token
length = []
for comment in comments:
    length.append(len(comment.split()))
max(length)

2222

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Tokenize the train comments
Word_tokenizer = Tokenizer()
Word_tokenizer.fit_on_texts(mearged_comment)
vocab_size = len(Word_tokenizer.word_index) + 1
# Word_tokenizer.num_words = 100000

Using TensorFlow backend.


In [11]:
# process tokens in training set
#encode the train tokens to sequence
sequences_train = Word_tokenizer.texts_to_sequences(train_comments)
#pading the sequence
x_train_seq = pad_sequences(sequences_train, maxlen=2250)
print('Shape of data tensor:', x_train_seq.shape)

Shape of data tensor: (978039, 2250)


In [13]:
# process tokens in testing set
#encode the test tokens to sequence
sequences_test = Word_tokenizer.texts_to_sequences(test_comments)
#pading the sequence
x_test_seq = pad_sequences(sequences_test, maxlen=2250)
print('Shape of data tensor:', x_test_seq.shape)

Shape of data tensor: (243572, 2250)


In [14]:
# load the Glove embedding into memory
embeddings_index = {}
f = open('glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [22]:
# create a weight matrix for words in train comments
# consider maximum of 100000 from the vocabulary
num_words = 100000
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in Word_tokenizer.word_index.items():
#     if i >= num_words:
#         continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [17]:
embedding_matrix.shape

(188492, 100)

In [16]:
# create a weight matrix for words in test comments
# consider maximum of 100000 from the vocabulary
num_words = 100000
embedding_matrix_test = np.zeros((num_words, 100))
for word, i in Word_tokenizer_test.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_test[i] = embedding_vector

In [18]:
from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

seed = 10

In [23]:
# model #1: Convolution neural network with 1 layer 1 embedding layer(trainable)
# Activation layer : Sigmoid
# loss : binary_crossentropy
# optimizer : adam
model_cnn_1 = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=2250, trainable=True)
model_cnn_1.add(e)
model_cnn_1.add(Conv1D(64, 3, padding='same'))
model_cnn_1.add(Flatten())
model_cnn_1.add(Dropout(0.2))
model_cnn_1.add(Dense(128,activation='sigmoid'))
model_cnn_1.add(Dropout(0.2))
model_cnn_1.add(Dense(1,activation='sigmoid'))
model_cnn_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# summarize the model
print(model_cnn_1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2250, 100)         18849200  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2250, 64)          19264     
_________________________________________________________________
flatten_1 (Flatten)          (None, 144000)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 144000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               18432128  
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total para

In [25]:
#checkpoint
from keras.callbacks import ModelCheckpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [27]:
# fit the model
model_cnn_1.fit(x_train_seq, train_labels, validation_split=0.2, epochs=10, batch_size=512, callbacks=callbacks_list)

Train on 782431 samples, validate on 195608 samples
Epoch 1/10
782431/782431 [==============================] - 17851s 23ms/step - loss: 0.5736 - acc: 0.6982 - val_loss: 0.5528 - val_acc: 0.7150

Epoch 00001: val_acc improved from -inf to 0.71503, saving model to weights.best.hdf5
Epoch 2/10
782431/782431 [==============================] - 13380s 17ms/step - loss: 0.5394 - acc: 0.7267 - val_loss: 0.5521 - val_acc: 0.7157

Epoch 00002: val_acc improved from 0.71503 to 0.71571, saving model to weights.best.hdf5
Epoch 3/10
782431/782431 [==============================] - 13418s 17ms/step - loss: 0.5124 - acc: 0.7463 - val_loss: 0.5543 - val_acc: 0.7160

Epoch 00003: val_acc improved from 0.71571 to 0.71604, saving model to weights.best.hdf5
Epoch 4/10
782431/782431 [==============================] - 13260s 17ms/step - loss: 0.4875 - acc: 0.7632 - val_loss: 0.5647 - val_acc: 0.7144

Epoch 00004: val_acc did not improve from 0.71604
Epoch 5/10
782431/782431 [==============================] 

In [30]:
#store the trained model.
from keras.models import model_from_json
#serialize model to JSON
model_json = model_cnn_1.to_json()
with open("model_cnn_1.json","w") as json_file:
    json_file.write(model_json)
#serialize weight to HDF4
model_cnn_1.save_weights("model_cnn_1.h5")
print("Saved model to disk")

Saved model to disk


In [37]:
#load json and create model
json_file = open('model_cnn_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

loaded_cnn_model = model_from_json(loaded_model_json)
#load weights into new model
loaded_cnn_model.load_weights("model_cnn_1.h5")
print("loaded model from disk")

loaded model from disk


In [33]:
# evaluate loadedmodel on test data
loaded_cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
score = loaded_cnn_model.evaluate(x_test_seq,test_labels)
print("%s: %.2f%%" %(loaded_cnn_model.metrics_names[1],score[1]*100))

243572/243572 [==============================] - 1102s 5ms/step
acc: 69.41%


In [36]:
#print the confusion matrix
predictions = loaded_cnn_model.predict_classes(x_test_seq)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print("Confusion Matrix")
print(confusion_matrix(test_labels, predictions))
print("Classification Report")
print(classification_report(test_labels, predictions))

Confusion Matrix
[[91129 29435]
 [45076 77932]]
Classification Report
              precision    recall  f1-score   support

           0       0.67      0.76      0.71    120564
           1       0.73      0.63      0.68    123008

   micro avg       0.69      0.69      0.69    243572
   macro avg       0.70      0.69      0.69    243572
weighted avg       0.70      0.69      0.69    243572

